In [1]:
import sys
sys.path.append("/scratch/group/csce435-f24/python-3.10.4/lib/python3.10/site-packages")
sys.path.append("/scratch/group/csce435-f24/thicket")
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import ScalarFormatter

import thicket as th

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
# Read all
tk = th.Thicket.from_caliperreader(glob("./*/**/*.cali", recursive=True))


(2/2) Creating Thicket: 100%|██████████| 203/203 [00:07<00:00, 26.63it/s]


In [3]:
print(tk.tree(metric_column="Avg time/rank"))

  _____ _     _      _        _   
 |_   _| |__ (_) ___| | _____| |_ 
   | | | '_ \| |/ __| |/ / _ \ __|
   | | | | | | | (__|   <  __/ |_ 
   |_| |_| |_|_|\___|_|\_\___|\__|  v2024.1.0

0.055 MPI_Comm_dup
0.000 MPI_Comm_free
0.000 MPI_Finalize
0.000 MPI_Finalized
0.000 MPI_Get_count
0.000 MPI_Initialized
0.000 MPI_Probe
0.000 MPI_Recv
0.000 MPI_Send
0.420 main
├─ 0.001 MPI_Barrier
├─ 0.000 MPI_Init
├─ 0.016 comm
│  ├─ 0.015 comm_large
│  │  ├─ 0.000 MPI_Gather
│  │  ├─ 0.001 MPI_Gatherv
│  │  └─ 0.013 MPI_Scatter
│  └─ 0.001 comm_small
│     ├─ 0.000 MPI_Allgather
│     └─ 0.000 MPI_Bcast
├─ 0.022 comp
│  ├─ 0.021 comp_large
│  └─ 0.000 comp_small
├─ 0.027 correctness_check
└─ 0.006 data_init_runtime

Legend (Metric: Avg time/rank Min: 0.00 Max: 0.42 indices: {'profile': np.int64(11797272)})
█ 0.38 - 0.42
█ 0.29 - 0.38
█ 0.21 - 0.29
█ 0.13 - 0.21
█ 0.04 - 0.13
█ 0.00 - 0.04

name User code    ◀  Only in left graph    ▶  Only in right graph



In [4]:
# Performance data
tk.metadata.transpose()

profile,11797272,18791168,45006884,70487119,110954104,130020471,180196807,183123200,206021162,251618419,281726832,285057149,295632864,338042823,373486299,377877913,434360546,462979056,511163290,514452733,517768604,531923235,536846300,606387808,636180519,646952715,654602214,712520227,736141061,769642718,840948235,848759067,854858957,862109346,868106744,889315274,930957566,933389875,936084854,966250942,1010736771,1017082770,1027847693,1063718608,1075534616,1119960351,1138294511,1183677650,1194339194,1215682319,1219610721,1227949030,1268235792,1357083356,1357095811,1404531810,1424045763,1464239510,1465772815,1525821676,1542370319,1569289301,1587632088,1590201369,1617383988,1640055675,1650833947,1651738848,1663062893,1665613787,1684819202,1685415425,1789272374,1863912484,1870148597,1896989755,1919242348,1947221742,1969398195,1989551443,1998629398,1999292682,2001293881,2026331959,2034415417,2056086545,2070363387,2087069541,2094757128,2105300391,2135196601,2151093919,2183599415,2190873370,2251963815,2254016061,2262214088,2288860266,2331394083,2377227623,2380126161,2386359086,2389228772,2432070606,2438215589,2466589888,2470463403,2490386517,2497991406,2520750039,2642992786,2662186035,2689006045,2698903712,2699393543,2728336266,2778900289,2809333259,2818949805,2833949445,2835873211,2854640669,2866777973,2871617377,2873932612,2879677834,2932264829,2932269110,2936115368,2992320786,3001283535,3004185666,3020387826,3022782358,3050692561,3095225457,3104154773,3119064468,3139296479,3151915748,3175800441,3244422841,3267222682,3269232241,3290826443,3326617168,3361165864,3361541609,3368971385,3377500288,3379654520,3386365736,3416802210,3438988046,3439998488,3440991389,3452381427,3469564611,3488771425,3517347613,3524213965,3564786690,3577497651,3584669097,3648549176,3659112656,3661786966,3662072482,3676886711,3680509426,3690094207,3740434903,3748967059,3776965750,3781577068,3798991149,3806099238,3845592860,3914232286,3941663623,3957806273,3959684669,3975165411,3975424958,4030109292,4090174516,4106810595,4111435940,4120395124,4120762697,4124377317,4142075814,4148046820,4161654763,4165242571,4171886278,4172207637,4189081943,4206040016,4222312880,4272665045,4276766168,4286411385,4291704417
cali.caliper.version,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0,2.11.0
mpi.world.size,4,64,512,8,64,4,128,512,256,8,2,32,512,16,8,4,16,64,4,8,64,32,256,256,2,8,128,32,16,64,256,4,16,32,256,16,512,2,2,16,16,4,256,128,16,128,512,8,32,2,64,4,2,8,2,2,8,4,1024,256,2,16,8,256,512,256,8,4,512,16,4,4,32,4,256,256,128,2,8,32,32,4,64,16,256,256,1024,512,64,2,2,512,128,256,64,512,512,128,256,1024,2,8,16,512,512,256,16,512

In [26]:
def strong_plot(time_col, size, cali_region, df):
    plt.figure(figsize=(15, 8))
    local_df = df[df["name"] == cali_region]
    local_df = local_df[local_df["input_size"] == size]
    local_df = local_df[local_df["algorithm"] == "merge"]
    local_df = local_df.drop_duplicates(subset=['num_procs', 'input_size', 'input_type'])
    local_df = local_df.sort_values(by=['input_size', 'num_procs'])
    
    for input_type, group in local_df.groupby('input_type'):
        plt.plot(group['num_procs'], group[time_col], marker='o', label=f'Input Type {input_type}')

    # Add labels and title
    plt.xlabel('Number of Processors (num_procs)')
    plt.ylabel(time_col)
    plt.title(f'Merge Sort {time_col} vs Number of Processors, Grouped by Input Size(size={size}, {cali_region})')
    
    plt.xscale('log', base=2)
    plt.xticks([2, 4, 8, 16, 32, 64, 128, 256, 512, 1024])
    plt.gca().xaxis.set_major_formatter(ScalarFormatter())
    
    # Add a legend
    plt.legend(title='Input Type')

    # Display the plot
    plt.grid(True)
    plt.show()

def weak_plot(time_col, input_type, cali_region, df):
    plt.figure(figsize=(15, 8))
    local_df = df[df["name"] == cali_region]
    local_df = local_df[local_df["algorithm"] == "merge"]
    local_df = local_df[~local_df["input_size"].isin([4,8, 256])]
    local_df = local_df[local_df["input_type"] == input_type]
    local_df = local_df.drop_duplicates(subset=['num_procs', 'input_size'])
    local_df = local_df.sort_values(by=['input_size', 'num_procs'])

    for input_size, group in local_df.groupby('input_size'):
        plt.plot(group['num_procs'], group[time_col], marker='o', label=f'Input Size {input_size}')

    # Add labels and title
    plt.xlabel('Number of Processors (num_procs)')
    plt.ylabel(time_col)
    plt.title(f'{time_col} vs Number of Processors, Grouped by Input Size({input_type}, {cali_region})')
    
    plt.xscale('log', base=2)
    plt.xticks([2, 4, 8, 16, 32, 64, 128, 256, 512, 1024])
    plt.gca().xaxis.set_major_formatter(ScalarFormatter())
    
    # Add a legend
    plt.legend(title='Input Size')

    # Display the plot
    plt.grid(True)
    plt.show()
    
def strong_plot_total_time(time_col, size, df):
    plt.figure(figsize=(15, 8))
    local_df = df[df["name"].isin(["comm", "comp"])]
    local_df = local_df[local_df["algorithm"] == "merge"]
    df_sum = local_df.groupby(['num_procs', 'input_size', 'input_type'], as_index=False).agg({time_col: 'sum'})

    # Filter by input size (assuming 'size' is defined globally or passed as an argument)
    df_sum = df_sum[df_sum['input_size'] == size]

    # Sort the values for better plotting
    df_sum = df_sum.sort_values(by=['input_size', 'num_procs'])

    for input_type, group in df_sum.groupby('input_type'):
        plt.plot(group['num_procs'], group[time_col], marker='o', label=f'Input Type {input_type}')

    # Add labels and title
    plt.xlabel('Number of Processors (num_procs)')
    plt.ylabel(time_col)
    plt.title(f'Merge Sort Total Time vs Number of Processors, Grouped by Input Size(size={size})')

    plt.xscale('log', base=2)
    plt.xticks([2, 4, 8, 16, 32, 64, 128, 256, 512, 1024])
    plt.gca().xaxis.set_major_formatter(ScalarFormatter())
    
    # Add a legend
    plt.legend(title='Input Type')

    # Display the plot
    plt.grid(True)
    plt.show()   

    
def weak_plot_total_time(time_col, input_type, df):
    plt.figure(figsize=(15, 8))
    local_df = df[df["name"].isin(["comm", "comp"])]
    local_df = local_df[local_df["algorithm"] == "merge"]
    df_sum = local_df.groupby(['num_procs', 'input_size', 'input_type'], as_index=False).agg({time_col: 'sum'})
    
    df_sum = df_sum[~df_sum["input_size"].isin([4,8, 256])]
    df_sum = df_sum[df_sum["input_type"] == input_type]
    df_sum = df_sum.drop_duplicates(subset=['num_procs', 'input_size'])
    df_sum = df_sum.sort_values(by=['input_size', 'num_procs'])

    for input_size, group in df_sum.groupby('input_size'):
        plt.plot(group['num_procs'], group[time_col], marker='o', label=f'Input Size {input_size}')

    # Add labels and title
    plt.xlabel('Number of Processors (num_procs)')
    plt.ylabel(time_col)
    plt.title(f'Total Time vs Number of Processors, Grouped by Input Size({input_type})')
    
    plt.xscale('log', base=2)
    plt.xticks([2, 4, 8, 16, 32, 64, 128, 256, 512, 1024])
    plt.gca().xaxis.set_major_formatter(ScalarFormatter())
    
    # Add a legend
    plt.legend(title='Input Size')

    # Display the plot
    plt.grid(True)
    plt.show()
    
    
def single_strong_plot_total_time(time_col, size, df):
    plt.figure(figsize=(15, 8))
    local_df = df[df["name"].isin(["comm", "comp"])]
    df_sum = local_df.groupby(['num_procs', 'input_size', 'input_type'], as_index=False).agg({time_col: 'sum'})

    # Filter by input size (assuming 'size' is defined globally or passed as an argument)
    df_sum = df_sum[df_sum['input_size'] == size]
    df_sum = df_sum[df_sum['input_type'] == "Random"]

    # Sort the values for better plotting
    df_sum = df_sum.sort_values(by=['input_size', 'num_procs'])

    
    plt.plot(df_sum['num_procs'], df_sum[time_col], marker='o', color='darkorange')

    # Add labels and title
    plt.xlabel('Number of Processors (num_procs)')
    plt.ylabel(time_col)
    plt.title(f'Merge Sort Total Time vs Number of Processors InputSize(size={size})')


    # Display the plot
    plt.grid(True)
    plt.show()   
    
    
    
def speedup_plot(size, df):
    plt.figure(figsize=(15, 8))
    filtered_df = df[df['name'].isin(['comp', 'comm'])]
    filtered_df = filtered_df[filtered_df["input_size"] == size]
    filtered_df = filtered_df[filtered_df["algorithm"] == "merge"]
    aggregated_df = filtered_df.groupby(['num_procs', 'input_size', 'input_type']).agg(
        all_procs=('Max time/rank', 'sum'),
        single_proc=('Total time', 'sum')
    ).reset_index()

    aggregated_df['speedup'] = aggregated_df['single_proc'] / aggregated_df['all_procs']
    aggregated_df.sort_values(by=['input_size', 'num_procs'])
    
    for input_type, group in aggregated_df.groupby('input_type'):
        plt.plot(group['num_procs'], group['speedup'], marker='o', label=f'Input Type {input_type}')
    
    plt.xlabel('Number of Processors (num_procs)')
    plt.ylabel('Speedup')
    plt.title(f'Speedup vs Number of Processors, Grouped by Input Size(size={size})')
    
    plt.xscale('log', base=2)
    plt.xticks([2, 4, 8, 16, 32, 64, 128, 256, 512, 1024])
    plt.gca().xaxis.set_major_formatter(ScalarFormatter())
    
    plt.legend(title='Input Size')
    
    plt.grid(True)
    plt.show() 
    
    
def speedup_plot_by_algorithm(size,input_type, df):
    input_types_list = []
    if (input_type == "Random"):
        input_types_list = ["Random", "random"]
    elif (input_type == "ReverseSorted"):
        input_types_list = ["ReverseSorted", "reversed"]
    elif (input_type == "Sorted"):
        input_types_list = ["Sorted", "sorted"]
    elif (input_type == "1_perc_perturbed"):
        input_types_list = ["1_perc_perturbed", "perturbed"]
        
        
    plt.figure(figsize=(15, 8))
    filtered_df = df[df['name'].isin(['comp', 'comm'])]

    filtered_df = filtered_df[filtered_df["input_size"] == size]
    filtered_df = filtered_df[filtered_df["input_type"].isin(input_types_list)]
    filtered_df = filtered_df.drop_duplicates(subset=['num_procs', 'input_size', 'input_type', 'algorithm', 'name'])
    aggregated_df = filtered_df.groupby(['num_procs', 'input_size', 'input_type','algorithm']).agg(
        all_procs=('Max time/rank', 'sum'),
        single_proc=('Total time', 'sum')
    ).reset_index()

    aggregated_df['speedup'] = aggregated_df['single_proc'] / aggregated_df['all_procs']
    aggregated_df.sort_values(by=['input_size', 'num_procs'])

    for algorithm, group in aggregated_df.groupby('algorithm'):
        plt.plot(group['num_procs'], group['speedup'], marker='o', label=f'{algorithm}')
    
    plt.xlabel('Number of Processors (num_procs)')
    plt.ylabel('Speedup')
    plt.title(f'Speedup vs Number of Processors, Grouped by Algorithm (input_size = {size}, input_type ={input_type})')
    
    plt.xscale('log', base=2)
    plt.xticks([2, 4, 8, 16, 32, 64, 128, 256, 512, 1024])
    plt.gca().xaxis.set_major_formatter(ScalarFormatter())
    
    plt.legend(title='Algorithm')
    
    plt.grid(True)
    plt.show() 

def speedup_plot_by_algorithm1(size,input_type, df):
    plt.figure(figsize=(15, 8))
    filtered_df = df[df['name'].isin(['comp', 'comm'])]
    filtered_df = filtered_df[filtered_df["input_size"] == size]
    filtered_df = filtered_df[filtered_df["input_type"] == input_type]
    filtered_df = filtered_df.drop_duplicates(subset=['num_procs', 'input_size', 'input_type', 'algorithm'])
    aggregated_df = filtered_df.groupby(['num_procs', 'input_size', 'input_type','algorithm']).agg(
        all_procs=('Max time/rank', 'sum'),
        single_proc=('Total time', 'sum')
    ).reset_index()
    
    aggregated_df['speedup'] = aggregated_df['single_proc'] / aggregated_df['all_procs']
    aggregated_df.sort_values(by=['input_size', 'num_procs'])

    for algorithm, group in aggregated_df.groupby('algorithm'):
        plt.plot(group['num_procs'], group['speedup'], marker='o', label=f'{algorithm}')
    
    plt.xlabel('Number of Processors (num_procs)')
    plt.ylabel('Speedup')
    plt.title(f'Speedup vs Number of Processors, Grouped by Algorithm , input_type={input_type})')
    
    plt.legend(title='Algorithm')
    
    plt.grid(True)
    plt.show()

In [27]:
    speedup_plot(2**16, tk.dataframe)
    speedup_plot(2**18, tk.dataframe)
    speedup_plot(2**20, tk.dataframe)
    speedup_plot(2**22, tk.dataframe)
    speedup_plot(2**24, tk.dataframe)
    speedup_plot(2**26, tk.dataframe)
    speedup_plot_by_algorithm(2**24, "Random", tk.dataframe)

KeyError: 'input_size'

<Figure size 1080x576 with 0 Axes>